# Packages

In [56]:
import pandas as pd
import requests
import bs4
import urllib
import time

# Scraping

In [57]:
base_url_individuals = 'https://insightcrime.org/criminal-actors/page/{}/?filter=personalities&country=0&orderby'
pages_individuals = 8

base_url_groups = 'https://insightcrime.org/criminal-actors/page/{}/?filter=armed_groups&country=0&orderby'
pages_groups = 9

In [58]:
individuals_list = []
groups_list = []

In [59]:
for page in range(1,pages_groups+1):
    
    url = base_url_groups.format(page)
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text,'lxml')
    
    boxes = soup.select('.page-general__news-list-criminals .box h2 a')
    page_list = [box.get_text(strip=True) for box in boxes]
    groups_list += page_list

In [60]:
for page in range(1,pages_individuals+1):
    
    url = base_url_individuals.format(page)
    res = requests.get(url)
    soup = bs4.BeautifulSoup(res.text,'lxml')
    
    boxes = soup.select('.page-general__news-list-criminals .box h2 a')
    page_list = [box.get_text(strip=True) for box in boxes]
    individuals_list += page_list

# Cleaning and export

## Individuals

In [61]:
individuals_list = [name.split(',') for name in individuals_list]
individuals_list = [[sublist[0], sublist[1].replace(' alias ', '').replace("'",'').replace('‘', '').replace('’', ''), sublist[2].replace(' or the ','').replace("'",'')] if len(sublist) == 3 
              else [sublist[0], sublist[1].replace(' alias ', '').replace("'",'').replace('‘', '').replace('’', ''), ''] if len(sublist) == 2
              else [sublist[0], '', '']
              for sublist in individuals_list]

In [62]:
df_individuals = pd.DataFrame(individuals_list).rename(columns={0:'name',1:'alias_1',2:'alias_2'})

In [63]:
dict_corr_names = {'Miguel Angel Treviño':'Miguel Angel Treviño Morales',
                  'Diego Fernando Murillo':'Diego Fernando Murillo Bejarano',
                  'Luis E. Calle Serna':'Luis Enrique Calle Serna'}
hector manuel beltran leyva

for name, correction in dict_corr_names.items():
    df_individuals.loc[df_individuals.name==name,'name'] = correction

In [64]:
def get_first_name(name):
    if len(name.split())==4:
        first_name = name.split()[0] + ' ' + name.split()[1]
    else:
        first_name = name.split()[0]
    return first_name

def get_last_name(name):
    if len(name.split())<=2:
        last_name = name.split()[-1]
    else:
        last_name = name.split()[-2] + ' ' + name.split()[-1]
    return last_name

In [65]:
df_individuals['first_name'] = df_individuals['name'].apply(get_first_name)
df_individuals['family_name'] = df_individuals['name'].apply(get_last_name)

We necessarily need to complement with visual inspection and make some corrections:

In [66]:
dict_corr_first_family_names = {'César Emilio Peralta':['César Emilio','Peralta'],
             'Jobanis de Jesús Ávila Villadiego':['Jobanis de Jesús','Ávila Villadiego'],
             'Leider Johani Noscue':['Leider Johani','Noscue'],
             'Walter Patricio Arizala':['Walter Patricio','Arizala'],
             'Henry de Jesús López':['Henry de Jesús','López'],
             'Horst Walther Overdick':['Horst Walther','Overdick'],
             'Dairo Antonio Úsuga':['Dairo Antonio','Úsuga'],
             'Juan de Dios Úsuga':['Juan de Dios','Úsuga'],
             'Juan Orlando Hernández':['Juan Orlando','Hernández'],
             'Ariel Máximo Cantero':['Ariel Máximo','Cantero'],
             'Pedro Oliverio Guerrero':['Pedro Oliverio','Guerrero']}

In [67]:
for name, corrections in dict_corr_first_family_names.items():
    df_individuals.loc[df_individuals.name==name,'first_name'] = corrections[0]
    df_individuals.loc[df_individuals.name==name,'family_name'] = corrections[1]

"Rastrojos" is actually a group and Juan Orlando Hernández was the president of Honduras

In [68]:
df_individuals = df_individuals[(df_individuals.name != 'Rastrojos') & (df_individuals.name != 'Juan Orlando Hernández')]

In [69]:
df_individuals.to_csv('_aux/Individuals.csv',index=False)